In [71]:
####################################################
## Exploratory Data Analysis of NOLA Police Calls ##
## @brett | 09-05-2021                            ##
####################################################

# Getting Data Ready

In [72]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.lines import Line2D
import imageio

# Change some settings
pd.options.display.max_rows = 999

# Import Data
last_year = pd.read_csv('./../data/Calls_for_service_2020.csv')
this_year = pd.read_csv('./../data/Calls_for_service_2021.csv')

# Merge into one dataset
data = this_year.append(last_year).reset_index().drop('index',axis=1)

# Extract Date from Timestamp
data['DateCreate'] = data['TimeCreate'].apply(lambda x: datetime.strptime(x[:10],'%m/%d/%Y'))

# Constants
CRS = "EPSG:4326"

In [73]:
# Read New Orleans Shapefile available here (https://portal-nolagis.opendata.arcgis.com/datasets/neighborhood-statistical-areas/explore)
no = gpd.read_file('./../data/Neighborhood_Statistical_Areas.zip')
no = no.to_crs(epsg=4326)

In [74]:
def get_latlon(s, lat=True):
    '''
    Helper Function to extract lat/lon from string
    '''
    try:
        t = s.replace('(','').replace(')','').split(' ')
        x = t[1]
        y = t[2]
        if(lat):
            return(float(x))
        else:
            return(float(y))
    except:
        return(None)

In [75]:
data['lat'] = data.Location.apply(lambda x: get_latlon(x, True))
data['lon'] = data.Location.apply(lambda x: get_latlon(x, False))

In [76]:
data = data[data.lat!=0]
data = data.reset_index()

color_mapping={
'AREA CHECK':'#003f5c'
,'COMPLAINT OTHER':'#2f4b7c'
,'DISTURBANCE (OTHER)':'#665191'
,'BUSINESS CHECK':'#a05195'
,'BURGLAR ALARM, SILENT':'#d45087'
,'RETURN FOR ADDITIONAL INFO':'#f95d6a'
,'SUSPICIOUS PERSON':'#ff7c43'
,'OTHER':'#ffa600'
}

In [77]:
data['TypeTextCleaned']= data.TypeText.apply(lambda x: x if x in color_mapping.keys() else 'OTHER')

In [78]:
geometry = gpd.points_from_xy(data.lat, data.lon, crs=CRS)
geo_df = gpd.GeoDataFrame(data, crs=CRS, geometry=geometry)

In [92]:
bounding_box = no.envelope
total_no = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])
intersections = gpd.overlay(geo_df, total_no, how='intersection')

In [10]:
# IGNORE_LIST = ['LAKE CATHERINE', 'VILLAGE DE LEST', 'VIAVANT - VENETIAN ISLES', 'NEW AURORA - ENGLISH TURN']
# no[~no.GNOCDC_LAB.isin(IGNORE_LIST)]

In [11]:
dates = [str(x)[:10] for x in data.DateCreate.unique()]
dates.sort()

In [107]:
in_bounds = geo_df[(geo_df.lat<=-89.6067156925987)&(geo_df.lat>=-90.12815362726941)&(geo_df.lon>=29.86561020508644)&(geo_df.lon<=30.17481060856237)]
for i, r in enumerate(dates):
    fig,ax = plt.subplots(figsize = (15,15))
    no.plot(ax = ax,color="#ECECEC", edgecolor="#D4D4D4")
    in_bounds[in_bounds.DateCreate==r]['geometry'].plot(ax=ax, color=geo_df['TypeTextCleaned'].map(color_mapping), alpha = 0.5)
    handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor=v, label=k, markersize=8) for k, v in color_mapping.items()]
    ax.legend(title='Incident Type', handles=handles, loc='upper right')
    ax.text(0.95, 0.05,r,
         horizontalalignment='center',
         verticalalignment='center',
         transform = ax.transAxes)
    ax.axis('off')
    plt.savefig(f'./../visualizations/raw/{i}.png')
    plt.close(fig)

In [108]:
with imageio.get_writer('./../visualizations/final/temporal.gif', mode='I') as writer:
    for filename in ['./../visualizations/raw/' + str(i) + '.png' for i, r in enumerate(dates)]:
        image = imageio.imread(filename)
        writer.append_data(image)